# [모듈 2.1] 리패키징 람다 싱글 스텝 

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 **약 5분** 소요 됩니다.

- 0. SageMaker Model Building Pipeline 개요
- 1. 파이프라인 변수 및 환경 설정
- 2. 파이프라인 스텝 단계 정의

    - (1) 모델 승인 상태 변경 람다 스텝    
    - (2) 배포할 세이지 메이커 모델 스텝 생성
    - (3) 모델 앤드 포인트 배포를 위한 람다 스텝 생성    
    
- 3. 모델 빌딩 파이프라인 정의 및 실행
- 4. Pipleline 캐싱 및 파라미터 이용한 실행
- 5. 정리 작업
    
---

# 1. 파이프라인 변수 및 환경 설정



In [34]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('./src')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker.Session().default_bucket()

print("bucket: ", bucket)
print("role: ", role)

sm_client = boto3.client('sagemaker', region_name=region)

%store -r model_package_group_name

bucket:  sagemaker-us-east-1-057716757052
role:  arn:aws:iam::057716757052:role/mlops-blog-ncf-gsmoon


In [44]:
%store -r inference_docker_image
# %store -r artifact_path
%store -r byom_artifact
%store -r bucket
%store -r prefix
%store -r artifact_path

In [45]:
byom_artifact

's3://sagemaker-us-east-1-057716757052/sm2fraud/new2train2model/model.tar.gz'

---
# 2. 패키징 코드
- 패키징할 폴더 생성
- 추론 코드의 폴더 지정 후에 패키징 폴더에 복사
- 패키징 폴더를 source.tar.gz 압축 후에 S3에 업로딩
- 



## 2.1. src 폴더를 source.tar.gz 로 압축

In [46]:
import os

package_dir = 'code_pkg'
os.makedirs(package_dir, exist_ok=True)


In [47]:
code_dir = '../src'

In [48]:
%%sh -s {package_dir} {code_dir}

package_dir=$1
code_dir=$2

cd $package_dir # 폴더 생성
echo $PWD
rm -rf ./*
cp -r $code_dir/*.py  .  # src py 모두 카피
cp -r $code_dir/*.txt  .  # src 파일 모두 카피
tar -czvf source.tar.gz * # model.tar.gz 파일 생성

/home/ec2-user/SageMaker/Neural-Collaborative-Filtering-On-SageMaker/3_MLOps/1_sm_pipeline/1_training_pipeline/code_pkg
back-iam_repackage_model_artifact.py
config.py
data_utils.py
evaluate.py
evaluation.py
iam_helper.py
iam_repackage_model_artifact.py
inference.py
inference_utils.py
lambda_helper.py
model.py
pipeline_util.py
repackage.py
requirements.txt
train_lib.py
train_metric.py
train_pipeline.py
train.py


## 2.2. S3 에 업로드

In [49]:
import sagemaker
sagemaker_session  = sagemaker.session.Session()
prefix='ncf/code'
bucket = sagemaker.session.Session().default_bucket()
print("bucket: ", bucket)

source_path = os.path.join(package_dir, 'source.tar.gz')
source_artifact = sagemaker_session.upload_data(source_path, bucket, prefix)
print("source_artifact: \n", source_artifact)

bucket:  sagemaker-us-east-1-057716757052
source_artifact: 
 s3://sagemaker-us-east-1-057716757052/ncf/code/source.tar.gz


### 모델 아피텍트 위치 확인

In [50]:
print("artifact_path: \n", artifact_path)

artifact_path: 
 s3://sagemaker-us-east-1-057716757052/pytorch-training-2022-10-14-13-19-13-618/output/model.tar.gz


# 3. 세이지 메이커 파이프라인 생성

## 3.1. 모델 빌딩 파이프라인 변수 생성

파이프라인에 인자로 넘길 변수는 아래 크게 3가지 종류가 있습니다.
- 모델 레지스트리에 모델 등록시에 모델 승인 상태 값    


In [51]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)


model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)


## 3.2. 파이프라인 스텝 단계 정의

### 3.2.1 람다 스텝 정의

####  람다 Role 생성

In [52]:
from src.iam_helper import create_lambda_role

lambda_role = create_lambda_role("lambda-deployment-role")
print("lambda_role: \n", lambda_role)

Using ARN from existing role: lambda-deployment-role
lambda_role: 
 arn:aws:iam::057716757052:role/lambda-deployment-role


#### 람다 스텝 정의

In [54]:
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)

# import time 
# current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())
# function_name = "sagemaker-lambda-step-approve-model-deployment-" + current_time
function_name = "sagemaker-lambda-step-repackage-model-artifact"

print("function_name: \n", function_name)
# Lambda helper class can be used to create the Lambda function
func_repackage_model = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="src/iam_repackage_model_artifact.py",
    handler="iam_repackage_model_artifact.lambda_handler",
)

output_param_1 = LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)
output_param_3 = LambdaOutput(output_name="S3_Model_URI", output_type=LambdaOutputTypeEnum.String)

step_approve_lambda = LambdaStep(
    name="LambdaRepackageStep",
    lambda_func=func_repackage_model,
    inputs={
        "source_path" : source_artifact,
        "model_path": artifact_path,
        "bucket" : bucket,
        "prefix" : "ncf/repackage/model"
    },
    outputs=[output_param_1, output_param_2, output_param_3],
)


# 4.모델 빌딩 파이프라인 정의 및 실행
위에서 정의한 아래의 4개의 스텝으로 파이프라인 정의를 합니다.
-     steps=[step_process, step_train, step_create_model, step_deploy],
- 아래는 약 1분 정도 소요 됩니다.  이후 아래와 같이 실행 결과를 스튜디오에서 확인할 수 있습니다.


- ![deployment-pipeline.png](img/deployment-pipeline.png)

In [55]:
from sagemaker.workflow.pipeline import Pipeline

project_prefix = 'ncf-pipeline-nb'

pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        model_approval_status,        
    ],

    
  steps=[step_approve_lambda],
#   steps=[step_approve_lambda, step_model_create, step_deploy_lambda],    
)



### 파이프라인을 SageMaker에 제출하고 실행하기 


In [56]:
import json

definition = json.loads(pipeline.definition())
# print(" definition : \n", definition)
pipeline.upsert(role_arn=role)
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

실행이 완료될 때까지 기다립니다.

In [59]:
execution.wait()

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

In [60]:
execution.list_steps()

[{'StepName': 'LambdaRepackageStep',
  'StartTime': datetime.datetime(2022, 10, 24, 6, 44, 10, 718000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 10, 24, 6, 44, 22, 653000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'Lambda': {'Arn': 'arn:aws:lambda:us-east-1:057716757052:function:sagemaker-lambda-step-repackage-model-artifact',
    'OutputParameters': [{'Name': 'body', 'Value': '"Repackaging is Done"'},
     {'Name': 'S3_Model_URI',
      'Value': 's3://sagemaker-us-east-1-057716757052/ncf/repackage/model/model.tar.gz'},
     {'Name': 'statusCode', 'Value': '200.0'}]}}}]

# 5. 정리 작업


In [62]:
# step_approve_lambda.properties.Outputs["other_key"]
# execution.list_steps()[1]['Metadata']['Lambda']['OutputParameters'][0]['Value']